#Silver to Gold: Building BI Ready Tables

In [0]:
#importing the required libraries
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql import Row

In [0]:
catalog_name = 'e-commerce'

##Products

In [0]:
#loading the data into dataframes
df_products = spark.table(f"`{catalog_name}`.silver.slv_products")
df_brands = spark.table(f"`{catalog_name}`.silver.slv_brands")
df_category = spark.table(f"`{catalog_name}`.silver.slv_category")

In [0]:
#creating views for the same dataframes
df_products.createOrReplaceTempView("v_products")
df_brands.createOrReplaceTempView("v_brands")
df_category.createOrReplaceTempView("v_category")

In [0]:
#spark sql
display(spark.sql("""select 
                    *
                   from v_products
                   limit 10"""))

In [0]:
display(spark.sql("""select 
                    *
                   from v_brands
                   limit 10"""))

In [0]:
display(spark.sql("""select 
                    *
                   from v_category
                   limit 10"""))

In [0]:
#setting a default catalog and also making sure we are on the right catalog
spark.sql(f"USE CATALOG `{catalog_name}`")

In [0]:
#Check the current catalog 
display(spark.sql("SELECT current_catalog()"))

In [0]:
%sql
--creating the final gold layer table joining these 3 silver layer tables

CREATE OR REPLACE TABLE gold.gld_dim_products AS

WITH brands_categories AS(
  SELECT 
    b.brand_code,
    b.brand_name,
    c.category_code,
    c.category_name
  FROM 
    v_brands b
  INNER JOIN
    v_category c
    ON b.category_code = c.category_code
) 
SELECT 
  p.product_id,
  p.sku,
  p.category_code,
  COALESCE(bc.category_name, "Not Available") AS category_name,
  p.brand_code,
  COALESCE(bc.brand_name, "Not Available") AS brand_name,
  p.color,
  p.size,
  p.material,
  p.weight_grams,
  p.length_cm,
  p.width_cm,
  p.height_cm,
  p.rating_count,
  p.file_name,
  p.ingest_timestamp
FROM v_products p
LEFT JOIN brands_categories bc
  ON p.brand_code = bc.brand_code;

In [0]:
display(spark.table("gold.gld_dim_products"))

##Customers

In [0]:
#mapping the states to the region for each country 

india_region = {
    "MH" : "West", "GJ" : "West", "RJ" : "West",
    "KA" : "South", "TN" : "South", "TS" : "South", "AP" : "South", "KL" : "South",
    "UP" : "North", "WB" : "North", "DL" : "North"
}

australia_region = {
    "VIC" : "SouthEast", "WA" : "West", "NSW" : "East", "QLD" : "NorthEast"
}

uk_region = {
    "ENG" : "England", "WLS" : "Wales", "NIR" : "Northern Ireland", "SCT" : "Scotland"
}

us_region = {
    "MA" : "NorthEast", "FL" : "South", "NJ" : "NorthEast", "CA" : "West",
    "NY" : "NorthEast", "TX" : "South"
}

uae_region = {
    "AUH" : "Abu Dhabi", "DU" : "Dubai", "SHJ" : "Sharjah"
}

singapore_region = {
    "SG" : "Singapore"
}

canada_region = {
    "BC" : "West", "AB" : "West", "ON" : "East", "QC" : "East", "NS" : "East", "IL" : "Other"
}

#Combine into a  master dictionary
country_state_map = {
    "India" : india_region,
    "Australia" : australia_region,
    "United Kingdom" : uk_region,
    "United States" : us_region,
    "United Arab Emirates" : uae_region,
    "Singapore" : singapore_region,
    "Canada" : canada_region
}

In [0]:
country_state_map

In [0]:
# Flatten country_state_map into a list of rows
rows = []
for country, states in country_state_map.items():
    for state_code, region in states.items():
        rows.append(Row(country = country, state = state_code, region = region))
rows[:10]

In [0]:
#create dataframe
df_region_mapping = spark.createDataFrame(rows)
df_region_mapping.show(truncate = False)

In [0]:
#bring in the silver table
df_silver = spark.table("silver.slv_customers")
display(df_silver.limit(5))

In [0]:
#doing the join 
df_gold = df_silver.join(df_region_mapping, on = ['country', 'state'], how = 'left')
df_gold = df_gold.fillna({"region" : "Other"})
display(df_gold.limit(5))

In [0]:
#writing it into catalog
df_gold.write.format("delta")\
             .mode("overwrite")\
             .option("mergeSchema", 'true')\
             .saveAsTable(f"`{catalog_name}`.gold.gld_dim_customers")

##Date/Calendar

In [0]:
df_silver = spark.table("silver.slv_calendar")
display(df_silver.limit(5))

In [0]:
# Parse the 'date' column from string to date type using the correct format
parsed_date = F.to_date(F.col("date"), "dd-MM-yyyy")

# Add the date_id column
df_gold = df_silver.withColumn("date_id", F.date_format(parsed_date, "yyyyMMdd").cast("int"))

# Add month name
df_gold = df_gold.withColumn("month_name", F.date_format(parsed_date, "MMMM"))

# Add is weekend indicator
# If 'day_name' is already present and correct, keep as is
# Otherwise, you may need to derive it from parsed_date
# For now, keep as is

# If you want to derive 'day_name' from parsed_date:
# df_gold = df_gold.withColumn("day_name", F.date_format(parsed_date, "EEEE"))

# Use 'day_name' column for weekend indicator
df_gold = df_gold.withColumn("is_weekend", F.when(F.col("day_name").isin("Saturday", "Sunday"), 1).otherwise(0))

display(df_gold.limit(5))

In [0]:
#setting the order right

desired_columns_order = ["date_id", "date", "year", "month_name", "day_name", "is_weekend", "quarter", "week", "_ingested_at", "_source_file"]

df_gold= df_gold.select(desired_columns_order)
display(df_gold.limit(5))

In [0]:
#write to catalog
df_gold.write.format("delta")\
       .mode("overwrite")\
       .option("mergeSchema", "true")\
       .saveAsTable(f"`{catalog_name}`.gold.gld_dim_date")